# Import Libraries

In [1]:
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt
import time
import mediapipe as mp

# 2. Keypoints using MP Holistic

In [2]:
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

In [3]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

In [4]:
def draw_styled_landmarks(image, results):
    # Draw face connections
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                             mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), 
                             mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                             ) 
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 

# 3. Extract Keypoint Values

In [5]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, lh, rh])

# 4. Setup Folders for Collection

In [29]:
# Path for exported data, numpy arrays
DATA_PATH = os.path.join('MP_Data') 

# Actions that we try to detect
actions= np.array(["thanks","zero","Amma","water","sorry","Rest","dolphin","ok","phone","akka","anna"])
# Thirty videos worth of data
no_sequences = 200

# Videos are going to be 30 frames in length
sequence_length = 50

In [7]:
#                                  #  uncommand when you are running it for first time
# for action in actions: 
#     for sequence in range(no_sequences):
#         try: 
#             os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
#         except:
#             pass

In [8]:
# cap = cv2.VideoCapture(0)
# # Set mediapipe model 
# with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
#     # NEW LOOP
#     # Loop through actions
#     for action in actions:
#         # Loop through sequences aka videos
#         for sequence in range(no_sequences):
#             # Loop through video length aka sequence length
#             for frame_num in range(sequence_length):

#                 # Read feed
#                 ret, frame = cap.read()

#                 # Make detections
#                 image, results = mediapipe_detection(frame, holistic)
#                 print(results)

#                 # Draw landmarks
#                 draw_styled_landmarks(image, results)
                
#                 # NEW Apply wait logic
#                 if frame_num == 0: 
#                     cv2.putText(image, 'STARTING COLLECTION', (120,200), 
#                                cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
#                     cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
#                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
#                     # Show to screen
#                     cv2.imshow('OpenCV Feed', image)
#                     cv2.waitKey(1000)
#                 else: 
#                     cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
#                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
#                     # Show to screen
#                     cv2.imshow('OpenCV Feed', image)
                
#                 # NEW Export keypoints
#                 keypoints = extract_keypoints(results)
#                 npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
#                 np.save(npy_path, keypoints)

#                 # Break gracefully
#                 if cv2.waitKey(10) & 0xFF == ord('q'):
#                     break
#                 if 0xFF == ord('p'):
#                     while True:
#                         print("waiting for 30 seconds")
#                         time.sleep(30)
#                         break
                    
#     cap.release()
#     cv2.destroyAllWindows()

In [9]:
# cap.release()
# cv2.destroyAllWindows()

# 6. Preprocess Data and Create Labels and Features

In [7]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [30]:
label_map = {label:num for num, label in enumerate(actions)}

In [31]:
label_map

{'thanks': 0,
 'zero': 1,
 'Amma': 2,
 'water': 3,
 'sorry': 4,
 'Rest': 5,
 'dolphin': 6,
 'ok': 7,
 'phone': 8,
 'akka': 9,
 'anna': 10}

In [32]:
sequences, labels = [], []
for action in actions[:]:
    for sequence in range(no_sequences):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [33]:
np.array(sequences).shape

(2200, 50, 1662)

In [34]:
np.array(labels).shape

(2200,)

In [35]:
X = np.array(sequences)

In [36]:
X.shape

(2200, 50, 1662)

In [37]:
y = to_categorical(labels).astype(int)

In [38]:
y.shape

(2200, 11)

In [39]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state=42)

In [40]:
X_train.shape

(1760, 50, 1662)

# 7. Build and Train CNN Neural Network

In [49]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Conv2D,MaxPool2D,Flatten
from tensorflow.keras.callbacks import TensorBoard

In [50]:
print(actions.shape)

(11,)


In [51]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [52]:
# callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=10)

In [53]:
# model = Sequential()

# model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(50,1662)))
# model.add(LSTM(128, return_sequences=True, activation='relu'))
# model.add(LSTM(64, return_sequences=False, activation='relu'))
# model.add(Dense(64, activation='relu'))
# model.add(Dense(32, activation='relu'))
# model.add(Dense(6, activation='softmax'))

In [54]:
model = Sequential()
model.add(Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=(50,1662,1)))
model.add(MaxPool2D(pool_size=(2, 2), strides=2))
model.add(Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding = 'valid'))
model.add(MaxPool2D(pool_size=(2, 2), strides=2))
model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding = 'same'))
model.add(MaxPool2D(pool_size=(2, 2), strides=2))
model.add(Flatten())
model.add(Dense(64,activation ="relu"))
model.add(Dense(32,activation ="relu"))
#model.add(Dropout(0.2))
# model.add(Dense(128,activation ="relu"))
#model.add(Dropout(0.3))
model.add(Dense(11,activation ="softmax"))

In [55]:
res = [.7, 0.2, 0.1]

In [56]:
actions[np.argmax(res)]

'thanks'

In [57]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [58]:
model.fit(X_train, y_train, epochs=500) 

Epoch 1/500
55/55 [==============================] - 64s 1s/step - loss: 2.3866 - categorical_accuracy: 0.1358
Epoch 2/500
55/55 [==============================] - 64s 1s/step - loss: 1.8992 - categorical_accuracy: 0.3517
Epoch 3/500
55/55 [==============================] - 64s 1s/step - loss: 1.4475 - categorical_accuracy: 0.5000
Epoch 4/500
55/55 [==============================] - 65s 1s/step - loss: 1.1368 - categorical_accuracy: 0.6205
Epoch 5/500
55/55 [==============================] - 66s 1s/step - loss: 0.9691 - categorical_accuracy: 0.6523
Epoch 6/500
55/55 [==============================] - 66s 1s/step - loss: 0.7959 - categorical_accuracy: 0.7148
Epoch 7/500
55/55 [==============================] - 65s 1s/step - loss: 0.6488 - categorical_accuracy: 0.7676
Epoch 8/500
55/55 [==============================] - 66s 1s/step - loss: 0.5496 - categorical_accuracy: 0.7926
Epoch 9/500
55/55 [==============================] - 65s 1s/step - loss: 0.4802 - categorical_accuracy: 0.8222
E

KeyboardInterrupt: 

In [59]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 48, 1660, 32)      320       
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 24, 830, 32)      0         
 2D)                                                             
                                                                 
 conv2d_7 (Conv2D)           (None, 22, 828, 128)      36992     
                                                                 
 max_pooling2d_7 (MaxPooling  (None, 11, 414, 128)     0         
 2D)                                                             
                                                                 
 conv2d_8 (Conv2D)           (None, 11, 414, 64)       73792     
                                                                 
 max_pooling2d_8 (MaxPooling  (None, 5, 207, 64)      

In [60]:
# 8. Make Predictions

In [61]:
res = model.predict(X_test)

14/14 [==============================] - 4s 246ms/step


In [62]:
actions[np.argmax(res[4])]

'ok'

In [63]:
actions[np.argmax(y_test[4])]

'ok'

# 9. Save Weights

In [64]:
model.save('cnn_all_model_tamil.h5')

In [65]:
model.load_weights('cnn_all_model_tamil.h5')

In [66]:
new_model = tf.keras.models.load_model('cnn_all_model_tamil.h5' , compile=False)

In [67]:
new_model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

# 10. Evaluation using Confusion Matrix and Accuracy

In [68]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [69]:
model = new_model

In [70]:
yhat = model.predict(X_test)

14/14 [==============================] - 3s 238ms/step


In [71]:
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [72]:
multilabel_confusion_matrix(ytrue, yhat)

array([[[398,   2],
        [  1,  39]],

       [[381,  13],
        [  6,  40]],

       [[376,  17],
        [  3,  44]],

       [[397,   4],
        [ 16,  23]],

       [[395,  11],
        [ 13,  21]],

       [[399,   1],
        [  3,  37]],

       [[399,   8],
        [  5,  28]],

       [[379,  21],
        [ 24,  16]],

       [[382,  21],
        [ 24,  13]],

       [[394,   2],
        [  3,  41]],

       [[399,   1],
        [  3,  37]]], dtype=int64)

In [73]:
accuracy_score(ytrue, yhat)

0.7704545454545455

# 11. Test in Real Time

In [74]:
from scipy import stats
# import pyttsx3

In [75]:
# pip install pyttsx3
# voice = pyttsx3.init()

In [76]:
actions= np.array(["thanks","zero","Amma","water","sorry","Rest","dolphin","ok","phone","akka","anna"])

In [77]:
# colors = [(245,117,16), (117,245,16), (16,117,245), (117,245,16), (16,117,245), (16,117,245),(117,245,16), (16,117,245), (16,117,245),(245,117,16), (117,245,16)]
colors =[(255, 0, 0) ,(0, 255, 0),(0, 0, 255),(255, 255, 0) ,(255, 255, 0) ,(0, 255, 255) ,(128, 0, 128) ,(255, 165, 0) ,(0, 128, 0),(128, 128, 128) ,(255, 127, 80)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame

In [82]:
# 1. New detection variables
sequence = []
sentence = []
predictions = []
threshold = 0.5

cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        print(results)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-50:]
        
        if len(sequence) == 50:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(np.argmax(res))
            print(actions[np.argmax(res)])
            predictions.append(np.argmax(res))
#             time.sleep(2)
            
        #3. Viz logic
            if np.unique(predictions[-10:])[0]==np.argmax(res): 
                if res[np.argmax(res)] > threshold: 
                    
                    if len(sentence) > 0: 
                        if actions[np.argmax(res)] != sentence[-1]:
                            sentence.append(actions[np.argmax(res)])
                    else:
                        sentence.append(actions[np.argmax(res)])

            if len(sentence) > 5: 
                sentence = sentence[-5:]

            # Viz probabilities
            image = prob_viz(res, actions, image, colors)
            
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        #resize image
        #image.resize(1920,1080)
        
        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

1
zero
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 64ms/step
1
zero
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 65ms/step
1
zero
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 65ms/step
2
Amma
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 64ms/step
1
zero
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 66ms/step
2
Amma
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 63ms/step
2
Amma
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 61ms/step
2
Amma
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 67ms/step
2
Amma
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [===

1/1 [==============================] - 0s 69ms/step
1
zero
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 75ms/step
1
zero
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 76ms/step
1
zero
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 73ms/step
1
zero
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 68ms/step
2
Amma
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 67ms/step
2
Amma
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 71ms/step
2
Amma
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 67ms/step
2
Amma
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 68ms/step
2
Amma
<class 'media

1/1 [==============================] - 0s 74ms/step
1
zero
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 75ms/step
1
zero
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 69ms/step
1
zero
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 72ms/step
1
zero
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 69ms/step
1
zero
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 74ms/step
1
zero
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 75ms/step
3
water
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 85ms/step
3
water
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 67ms/step
3
water
<class 'me

1/1 [==============================] - 0s 64ms/step
6
dolphin
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 64ms/step
8
phone
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 66ms/step
8
phone
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 69ms/step
8
phone
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 65ms/step
6
dolphin
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 67ms/step
6
dolphin
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 66ms/step
6
dolphin
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 68ms/step
6
dolphin
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 64ms/step
6


...

1/1 [==============================] - 0s 64ms/step
1
zero
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 65ms/step
1
zero
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 62ms/step
1
zero
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 64ms/step
1
zero
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 62ms/step
1
zero
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 64ms/step
1
zero
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 66ms/step
1
zero
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 68ms/step
1
zero
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 62ms/step
1
zero
<class 'media

1/1 [==============================] - 0s 65ms/step
3
water
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 68ms/step
3
water
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 66ms/step
3
water
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 69ms/step
3
water
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 69ms/step
3
water
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 67ms/step
3
water
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 69ms/step
3
water
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 67ms/step
3
water
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 68ms/step
3
water
<cla

1/1 [==============================] - 0s 70ms/step
9
akka
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 72ms/step
9
akka
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 71ms/step
9
akka
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 72ms/step
9
akka
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 74ms/step
9
akka
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 72ms/step
9
akka
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 68ms/step
9
akka
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 68ms/step
9
akka
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 69ms/step
9
akka
<class 'media

1/1 [==============================] - 0s 81ms/step
1
zero
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 77ms/step
1
zero
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 80ms/step
1
zero
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 77ms/step
1
zero
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 78ms/step
1
zero
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 77ms/step
3
water
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 78ms/step
3
water
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 79ms/step
9
akka
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 75ms/step
9
akka
<class 'med

1/1 [==============================] - 0s 77ms/step
5
Rest
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 78ms/step
5
Rest
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 80ms/step
5
Rest
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 80ms/step
5
Rest
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 75ms/step
5
Rest
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 78ms/step
5
Rest
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 80ms/step
5
Rest
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 79ms/step
5
Rest
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 78ms/step
5
Rest
<class 'media

In [ ]:
cap.release()
cv2.destroyAllWindows()

In [32]:
print(tf.__version__)

2.12.0
